In [1]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, SimpleRNN
from keras.layers.wrappers import TimeDistributed
import keras.callbacks
from keras import optimizers
import argparse
import pickle
from RNN_utils import *



Using TensorFlow backend.


In [2]:
DEFAULT_LAYERS = 2
DEFAULT_HIDDEN = 500
DEFAULT_DROPOUT = 0.0

layers = [1, 3]
hidden = [300, 700]
dropout = [0.15, 0.3]
DATA_DIR = './data/shakespeare_input.txt'
BATCH_SIZE = 30
SEQ_LENGTH = 50

num_epochs = 50

# Creating training data
X, y, VOCAB_SIZE, ix_to_char, char_to_ix = load_data(DATA_DIR, SEQ_LENGTH)

Data length: 4573338 characters
Vocabulary size: 67 characters


In [3]:
# Split testing data from training/validation data
train_split = 0.8
test_ind = int(round(train_split*len(X)))
X_test = X[test_ind:]
y_test = y[test_ind:]
X_train = X[:test_ind]
y_train = y[:test_ind]

In [4]:
def make_lstm_model(VOCAB_SIZE, num_layers, num_hidden, dropout):
    model = Sequential()
    model.add(LSTM(num_hidden, input_shape=(None, VOCAB_SIZE), return_sequences=True, dropout=dropout))
    for i in range(num_layers - 1):
      model.add(LSTM(num_hidden, return_sequences=True,dropout=dropout))
    model.add(TimeDistributed(Dense(VOCAB_SIZE)))
    model.add(Activation('softmax'))
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
    return model

In [5]:
def run_model(model, X, y, num_epochs=50, batch_size=30):
    early_stop1 = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=1, mode='auto')
    early_stop2 = keras.callbacks.EarlyStopping(monitor='loss',
                              min_delta=0,
                              patience=2,
                              verbose=1, mode='auto')
    history = model.fit(X, y, batch_size=batch_size, callbacks=[early_stop1, early_stop2], validation_split=0.2, epochs=num_epochs, verbose=1)
    return model, history.history

In [9]:
def save_model(model, history, num_layers, num_hidden, dropout):
    string = 'layers_{}_hidden_{}_dropout_{}_epoch_{}'.format(num_layers, num_hidden, dropout, len(history['loss']))
    with open('history'+string, 'wb') as file_pi:
        pickle.dump(history, file_pi)
    model.save_weights('weights'+string+'.hdf5'.format(num_layers, num_hidden, dropout, len(history['loss'])))

In [7]:
default_model = make_lstm_model(VOCAB_SIZE, DEFAULT_LAYERS, DEFAULT_HIDDEN, DEFAULT_DROPOUT)

In [8]:
default_model, default_history = run_model(default_model, X_train, y_train)
save_model(default_model, default_history, DEFAULT_LAYERS, DEFAULT_HIDDEN, DEFAULT_DROPOUT)

NameError: name 'default_model' is not defined

In [ ]:
for layer in layers:
    model = make_lstm_model(VOCAB_SIZE, layer, DEFAULT_HIDDEN, DEFAULT_DROPOUT)
    model, history = run_model(model, X_train, y_train)
    history['test_loss_predict'] = evaluate_loss(model, X_test)
    history['test_loss_eval'] = model.evaluate(X_test, y_test, batch_size=30)
    save_model(model, history, layer, DEFAULT_HIDDEN, DEFAULT_DROPOUT)
    

In [ ]:
for h in hidden:
    model = make_lstm_model(VOCAB_SIZE, DEFAULT_LAYERS, h, DEFAULT_DROPOUT)
    model, history = run_model(model, X_train, y_train)
    history['test_loss_predict'] = evaluate_loss(model, X_test)
    history['test_loss_eval'] = model.evaluate(X_test, y_test, batch_size=30)
    save_model(model, history, DEFAULT_LAYERS, h, DEFAULT_DROPOUT)
    

In [ ]:
for d in dropout:
    model = make_lstm_model(VOCAB_SIZE, DEFAULT_LAYERS, DEFAULT_HIDDEN, d)
    model, history = run_model(model, X_train, y_train)
    history['test_loss_predict'] = evaluate_loss(model, X_test)
    history['test_loss_eval'] = model.evaluate(X_test, y_test, batch_size=30)
    save_model(model, history, DEFAULT_LAYERS, DEFAULT_HIDDEN, d)
    